<a href="https://colab.research.google.com/github/Mdhasim-tech/Deep-Learning/blob/main/Fake_news_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Bidirectional,Dense,Dropout,GlobalMaxPool1D
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [ ]:
# 1) Load and label
fake=pd.read_csv("/content/drive/MyDrive/Datasets/News _dataset/Fake.csv")
true=pd.read_csv("/content/drive/MyDrive/Datasets/News _dataset/True.csv")

In [ ]:
fake['label']=1
true['label']=0

fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [ ]:
df=pd.concat([fake,true],ignore_index=True).sample(frac=1,random_state=42).reset_index(drop=True)
df.head(10)

,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",1
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",0
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",0
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",1
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",0
5,Paul Ryan Responds To Dem’s Sit-In On Gun Con...,"On Wednesday, Democrats took a powerful stance...",News,"June 22, 2016",1
6,AWESOME! DIAMOND AND SILK Rip Into The Press: ...,President Trump s rally in FL on Saturday was ...,Government News,"Feb 19, 2017",1
7,STAND UP AND CHEER! UKIP Party Leader SLAMS Ge...,He s been Europe s version of the outspoken Te...,left-news,"Mar 8, 2016",1
8,North Korea shows no sign it is serious about ...,WASHINGTON (Reuters) - The State Department sa...,worldnews,"December 13, 2017",0
9,Trump signals willingness to raise U.S. minimu...,(This version of the story corrects the figur...,politicsNews,"May 4, 2016",0


In [ ]:
# Basic preprocessing helper
def clean_text(s):
  if not isinstance(s,str):return ""
  s=s.lower()
  s=re.sub(r"http\S+|www\S+|https\S+", " ", s) # remove urls
  s=re.sub(r"\<.*?\>"," ",s)  # remove html tags
  s=re.sub(r"[^a-z0-9\s]", " ", s) # keeps alphanum and removes everthing else
  s=re.sub(r"\s+"," ",s).strip() # removes extra spaces
  return s

# use title + text
df['content']=(df['title'].fillna("")+" "+df['text'].fillna("")).apply(clean_text)



In [ ]:
df['content'][0]

'ben stein calls out 9th circuit court committed a coup d tat against the constitution 21st century wire says ben stein reputable professor from pepperdine university also of some hollywood fame appearing in tv shows and films such as ferris bueller s day off made some provocative statements on judge jeanine pirro s show recently while discussing the halt that was imposed on president trump s executive order on travel stein referred to the judgement by the 9th circuit court in washington state as a coup d tat against the executive branch and against the constitution stein went on to call the judges in seattle political puppets and the judiciary political pawns watch the interview below for the complete statements and note the stark contrast to the rhetoric of the leftist media and pundits who neglect to note that no court has ever blocked any presidential orders in immigration in the past or discuss the legal efficacy of the halt or the actual text of the executive order read more trum

In [ ]:
df.head()

,title,text,subject,date,label,content
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",1,ben stein calls out 9th circuit court committe...
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",0,trump drops steve bannon from national securit...
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",0,puerto rico expects u s to lift jones act ship...
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",1,oops trump just accidentally confirmed he leak...
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",0,donald trump heads for scotland to reopen a go...


In [ ]:


X = df["content"].values
y = df["label"].values

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,        # 20% goes to test
    random_state=42,      # reproducible results
    stratify=y            # keep label proportions the same in train/test
)

In [ ]:
df['content'].values
df['label'].values

array([1, 0, 0, ..., 0, 1, 1])

In [ ]:
# 4) Tokenize
max_words = 50000       # vocabulary size
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)


In [ ]:
tokenizer.word_index

{'<OOV>': 1,
 'the': 2,
 'to': 3,
 'of': 4,
 'a': 5,
 'and': 6,
 'in': 7,
 's': 8,
 'that': 9,
 'on': 10,
 'for': 11,
 'is': 12,
 'trump': 13,
 'it': 14,
 'he': 15,
 'said': 16,
 'with': 17,
 'was': 18,
 'as': 19,
 'his': 20,
 'by': 21,
 'has': 22,
 'be': 23,
 'not': 24,
 'have': 25,
 'from': 26,
 'this': 27,
 'at': 28,
 'are': 29,
 'who': 30,
 'they': 31,
 'an': 32,
 'i': 33,
 'we': 34,
 'but': 35,
 'u': 36,
 'president': 37,
 'would': 38,
 't': 39,
 'about': 40,
 'will': 41,
 'you': 42,
 'their': 43,
 'had': 44,
 'been': 45,
 'people': 46,
 'more': 47,
 'were': 48,
 'or': 49,
 'after': 50,
 'one': 51,
 'she': 52,
 'which': 53,
 'her': 54,
 'if': 55,
 'out': 56,
 'all': 57,
 'state': 58,
 'what': 59,
 'when': 60,
 'new': 61,
 'up': 62,
 'there': 63,
 'obama': 64,
 'also': 65,
 'its': 66,
 'no': 67,
 'clinton': 68,
 'over': 69,
 'house': 70,
 'reuters': 71,
 'donald': 72,
 'government': 73,
 'so': 74,
 'our': 75,
 'can': 76,
 'states': 77,
 'him': 78,
 'republican': 79,
 'just': 80,
 '

In [ ]:
tokenizer.texts_to_sequences(["Fake news spreads fast"])

[[660, 99, 12301, 2088]]

In [ ]:
tokenizer.texts_to_sequences(["Fake news spreads slowly"])

[[660, 99, 12301, 6197]]

In [ ]:
# padding
maxlen=300
X_train_seq=pad_sequences(tokenizer.texts_to_sequences(X_train),maxlen=maxlen,padding='post',truncating='post')
X_test_seq=pad_sequences(tokenizer.texts_to_sequences(X_test),maxlen=maxlen,padding='post',truncating='post')

In [ ]:
# Build model (train embedding from scratch)
embedding_dim=100
model=Sequential([
    Embedding(input_dim=max_words,output_dim=embedding_dim,input_length=maxlen,mask_zero=False),
    Bidirectional(LSTM(128,return_sequences=True)),
    GlobalMaxPool1D(),
    Dropout(0.5),
    Dense(64,activation='relu'),
    Dropout(0.3),
    Dense(1,activation="sigmoid")
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_3          │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train
history=model.fit(
    X_train_seq,
    y_train,
    validation_split=0.1, # 10% of training set used for validation
    epochs=8,
    batch_size=128
)

Epoch 1/8
253/253 ━━━━━━━━━━━━━━━━━━━━ 598s 2s/step - accuracy: 1.0000 - loss: 1.3930e-04 - val_accuracy: 0.9994 - val_loss: 0.0058
Epoch 2/8
253/253 ━━━━━━━━━━━━━━━━━━━━ 611s 2s/step - accuracy: 0.9999 - loss: 2.1642e-04 - val_accuracy: 0.9989 - val_loss: 0.0051
Epoch 3/8
253/253 ━━━━━━━━━━━━━━━━━━━━ 621s 2s/step - accuracy: 1.0000 - loss: 1.3195e-04 - val_accuracy: 0.9992 - val_loss: 0.0061
Epoch 4/8
253/253 ━━━━━━━━━━━━━━━━━━━━ 602s 2s/step - accuracy: 0.9999 - loss: 1.9576e-04 - val_accuracy: 0.9994 - val_loss: 0.0061
Epoch 5/8
253/253 ━━━━━━━━━━━━━━━━━━━━ 618s 2s/step - accuracy: 1.0000 - loss: 4.3072e-05 - val_accuracy: 0.9994 - val_loss: 0.0065
Epoch 6/8
226/253 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 1.0000 - loss: 3.4699e-05 